# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptEncoderOnlyClassifier, VanillaEncoderOnlyClassifier

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
KGE_DIMENSIONS = [2, 4, 8, 16] # Output Dimension of the GNN Encoder.

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

Using existing file ml-latest-small.zip
Extracting ./data\ml-latest-small.zip


generate llm dataset...
splitting LLM dataset


In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split
0,0,0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","user: 0, title: Toy Story (1995), genres: ['Ad...",val
1,0,2,Grumpier Old Men (1995),"[Comedy, Romance]","user: 0, title: Grumpier Old Men (1995), genre...",train
2,0,5,Heat (1995),"[Action, Crime, Thriller]","user: 0, title: Heat (1995), genres: ['Action'...",train
3,0,43,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]","user: 0, title: Seven (a.k.a. Se7en) (1995), g...",train
4,0,46,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]","user: 0, title: Usual Suspects, The (1995), ge...",train


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainers =    [GNNTrainer(movie_lens_loader.data, kge_dimension = kge_dimension) for kge_dimension in KGE_DIMENSIONS]

Device: 'cuda'
Device: 'cuda'
Device: 'cuda'
Device: 'cuda'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:
for gnn_trainer in gnn_trainers:
    print(gnn_trainer.kge_dimension)
    gnn_trainer.train_model(movie_lens_loader.gnn_train_data, 10)
    gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)


2


100%|██████████| 79/79 [00:04<00:00, 16.96it/s]



Validation AUC: 0.4849
4


100%|██████████| 79/79 [00:04<00:00, 17.22it/s]



Validation AUC: 0.5150
8


100%|██████████| 79/79 [00:04<00:00, 17.05it/s]



Validation AUC: 0.4983
16


100%|██████████| 79/79 [00:04<00:00, 16.90it/s]


Validation AUC: 0.4549


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
[gnn_trainer.get_embeddings(movie_lens_loader) for gnn_trainer in gnn_trainers]
movie_lens_loader.llm_df.head()

Computing embeddings for embedding dimension 2.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
vanilla_encoder_only_classifier = VanillaEncoderOnlyClassifier(movie_lens_loader.llm_df)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_encoder_only_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_encoder_only_classifier.train_model_on_data(dataset_vanilla, epochs=20, batch_size = 256)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
prompt_encoder_only_classifiers = [PromptEncoderOnlyClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=embedding_dimension, batch_size=256) for embedding_dimension, gnn_trainer in zip(KGE_DIMENSIONS, gnn_trainers)]

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
datasets_prompt = [movie_lens_loader.generate_prompt_embedding_dataset(prompt_encoder_only_classifier.tokenize_function, kge_dimension = prompt_encoder_only_classifier.kge_dimension) for prompt_encoder_only_classifier in prompt_encoder_only_classifiers]

We also train the model. This can be skipped if already done ones.

In [ ]:
[prompt_encoder_only_classifier.train_model_on_data(dataset_prompt, epochs = 20) for prompt_encoder_only_classifier, dataset_prompt in zip(prompt_encoder_only_classifiers, datasets_prompt)]